## Worldometer Site Web Scrapping

---

In [1]:
# Import libraries

import os
import re
import glob
import requests 
import numpy as np
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup

In [2]:
# Import dataset

link = 'https://www.worldometers.info/coronavirus/'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# Find table

thead = soup.find_all('thead')[-1]
head = thead.find_all('tr')
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

In [4]:
# Get the table contents

head_rows = []
body_rows = []

for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
print(head_rows)

[['#', 'Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical', 'Tot\xa0Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/\n1M pop\n', 'Population', 'Continent', '1 Caseevery X ppl', '1 Deathevery X ppl', '1 Testevery X ppl']]


In [5]:
# Save content in dataframe

df_wm = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

df_wm.tail(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
217,210,Solomon Islands,17,,,,10,,7,,24,,"4,500","6,471","695,441",Australia/Oceania,"40,908",,155
218,211,Saint Pierre Miquelon,16,,,,14,,2,,"2,769",,"3,196","553,037","5,779",North America,361,,2
219,212,Anguilla,15,,,,13,,2,,995,,"5,337","354,100","15,072",North America,"1,005",,3
220,213,Montserrat,13,,1,,12,,0,,"2,603",200,626,"125,350","4,994",North America,384,"4,994",8
221,214,Western Sahara,10,,1,,8,,1,,17,2,,,"604,799",Africa,"60,480","604,799",


In [6]:
# Drop unwanted rows & columns

df_wm = df_wm.iloc[8:, :-3].reset_index(drop=True)
df_wm = df_wm.drop('#', axis=1)
df_wm.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent
0,USA,"21,578,606",,"365,620",,"12,862,216",,"8,350,770","29,733","64,995","1,101","261,856,043","788,717","332,002,717",North America
1,India,"10,375,478",,"150,151",,"9,997,272",+771,"228,055","8,944","7,481",108,"177,463,405","127,950","1,386,975,123",Asia
2,Brazil,"7,812,007",,"197,777",,"6,963,407",,"650,823","8,318","36,618",927,"28,600,000","134,060","213,337,236",South America
3,Russia,"3,284,384",,"59,506",,"2,662,668",,"562,210","2,300","22,501",408,"92,000,000","630,280","145,966,823",Europe
4,UK,"2,774,479",,"76,305",,N/A,N/A,N/A,"1,847","40,759","1,121","57,144,355","839,496","68,069,826",Europe


In [7]:
# Rename & rearrange columns
df_wm.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']

df_wm = df_wm[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]

df_wm.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,"332,002,717","21,578,606",,"365,620",,"12,862,216",,"8,350,770","29,733","64,995","1,101","261,856,043","788,717"
1,India,Asia,"1,386,975,123","10,375,478",,"150,151",,"9,997,272",+771,"228,055","8,944","7,481",108,"177,463,405","127,950"
2,Brazil,South America,"213,337,236","7,812,007",,"197,777",,"6,963,407",,"650,823","8,318","36,618",927,"28,600,000","134,060"
3,Russia,Europe,"145,966,823","3,284,384",,"59,506",,"2,662,668",,"562,210","2,300","22,501",408,"92,000,000","630,280"
4,UK,Europe,"68,069,826","2,774,479",,"76,305",,N/A,N/A,N/A,"1,847","40,759","1,121","57,144,355","839,496"


In [8]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [9]:
# Addding 'WHO Region' & list of countries with missing "WHO Region"

df_wm['WHO Region'] = df_wm['Country/Region'].map(who_region)
df_wm[df_wm['WHO Region'].isna()]['Country/Region'].unique()

array(['French Polynesia', 'French Guiana', 'Réunion', 'Guadeloupe',
       'Martinique', 'Mayotte', 'Aruba', 'CAR', 'Curaçao',
       'Channel Islands', 'Gibraltar', 'Sint Maarten', 'Saint Martin',
       'Turks and Caicos', 'Diamond Princess', 'Faeroe Islands',
       'Isle of Man', 'Cayman Islands', 'Caribbean Netherlands',
       'St. Barth', 'Brunei ', 'St. Vincent Grenadines',
       'British Virgin Islands', 'Macao', 'New Caledonia',
       'Falkland Islands', 'Saint Pierre Miquelon', 'Anguilla',
       'Montserrat'], dtype=object)

In [10]:
# Data corrections

for col in df_wm.columns[2:]:
    # replace comma with empty string
    df_wm[col] = df_wm[col].str.replace('[,+ ]', '', regex=True)
    # replace 'N/A' with empty string
    df_wm[col] = df_wm[col].str.replace('N/A', '', regex=False)

# replace empty strings with np.nan
df_wm = df_wm.replace('', np.nan)

df_wm.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,332002717,21578606,NaN,365620,NaN,12862216,NaN,8350770,29733,64995,1101,261856043,788717,Americas
1,India,Asia,1386975123,10375478,NaN,150151,NaN,9997272,771,228055,8944,7481,108,177463405,127950,South-EastAsia
2,Brazil,South America,213337236,7812007,NaN,197777,NaN,6963407,NaN,650823,8318,36618,927,28600000,134060,Americas
3,Russia,Europe,145966823,3284384,NaN,59506,NaN,2662668,NaN,562210,2300,22501,408,92000000,630280,Europe
4,UK,Europe,68069826,2774479,NaN,76305,NaN,NaN,NaN,NaN,1847,40759,1121,57144355,839496,Europe


In [11]:
# Save as "csv" file

df_wm.to_csv('worldometer_data.csv', index=False)